# 앙상블 학습 ensemble learning
>정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘  
더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 머신러닝 알고리즘

**정형 데이터**  
가지런히 정리되어 있는 데이터  
CSV, 엑셀, SQL데이터베이스 타입

**비정형 데이터**  
텍스트 데이터, 사진, 디지털 음악 등

## 랜덤 포레스트 Random Forest
>앙상블 학습의 대표 주자 중 하나로 안정적인 성능 덕분에 널리 사용되고 있다.  
결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만든다.  
각 트리를 부트스트램 샘플로 훈련시킨다.

### 부트스트랩 샘플 boostrap sample
>훈련 데이터에서 램덤하게 샘플을 추출하여 훈련 데이터를 만들 때 중복을 허용하여 만든 샘플  
기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만든다.

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [2]:
data = wine.iloc[:,:3].to_numpy()
target = wine['class'].to_numpy()
print(data[:5])
print(target[:5])

[[9.4  1.9  3.51]
 [9.8  2.6  3.2 ]
 [9.8  2.3  3.26]
 [9.8  1.9  3.16]
 [9.4  1.9  3.51]]
[0. 0. 0. 0. 0.]


In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [4]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
import numpy as np

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [5]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


결정 트리와 비교했을 때 두 번째 특성의 중요도가 감소하고 다른 두 개의 특성의 중요도가 올라갔다.  
램덤 포레스트가 특성의 일부를 램덤하게 선택하여 결정 트리를 훈련하기 때문에 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다.

### OOB(out of bag) 샘플
부트스트랩 샘플에 포함되지 않고 남는 샘플  
OOB 샘플 점수를 사용하면 교차 검증을 대신할 수 있어서 결과적으로 훈련 세트에 더 많은 샘플을 사용할 수 있다.


In [6]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라 트리 Extra Trees
부트스트랩 샘플을 사용하지 않는 램덤 포레스트  
램덤하게 노드를 분할하기 때문에 빠른 계산 속도가 빠르고 과대적합이 감소한다.

In [7]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [8]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅 gradient boosting
깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법  
깊이가 얕은 결정트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있다.

In [9]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [10]:
gb = GradientBoostingClassifier(n_estimators=500, # 결정트리개수 default:100
                                learning_rate=0.2, # 학습률 default:0.1
                                random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [11]:
gb.fit(train_input,train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


그레이디언트 부스팅은 순서대로 트리를 추가하기 때문에 훈련속도가 느리고 n_jobs 매개변수가 없다.

## 히스토그램 기반 그레이디언트 부스팅 Histogram-based Gradient Boosting
입력 특성을 256개의 구간으로 나누고 구간 중에서 하나를 떼어 놓고 누락된 값을 빼놓는다.  
**따라서 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없다.**

In [14]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [16]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats=10, # default : 5
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [18]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [19]:
hgb.score(test_input,test_target)

0.8723076923076923

### 그레이디언트 부스팅 알고리즘을 구현한 라이브러리
XGBoost : https://xgboost.ai/ & https://xgboost.readthedocs.io/en/latest  
LightGBM(마이크로소프트) : https://github.com/microsoft/LightGBM & https://lightgbm.readthedocs.io/en/latest

In [23]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [24]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
